In [1]:
import os
import sys
import time
import numpy as np
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from models.pytorch_vggish import pretrained

from src.data import load_annotation, load_data
from src.seq2seq.train import train_test_split, train, test
from src.seq2seq.model import LSTMClassifier

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [7]:
# configuration
RAWDATA_PATH = "../data/raw"
DATAFRAME_PATH = os.path.join(RAWDATA_PATH, "data_frames")
ANNOTATION_PATH = "../data/processed/Annotation.csv"
CATEGORY = "Urination"
THRESHOLD = 0.3
NUM_EPOCHS = 5

PYTORCH_MODEL = pretrained.make_pytorch_vggish(path = "../data/processed/pytorch_vggish.pth")
ANNOTATIONS = load_annotation.get_annotation(ANNOTATION_PATH)
USER_IDS = load_annotation.get_complete_ids(ANNOTATION_PATH, CATEGORY)[:80]
TRAIN_IDS, TEST_IDS = train_test_split(USER_IDS)

seq2seq_train_config = {
    'USER_IDS': TRAIN_IDS,
    'DATAFRAME_PATH': DATAFRAME_PATH,
    'CATEGORY': CATEGORY,
    'ANNOTATIONS': ANNOTATIONS,
    'PYTORCH_MODEL': PYTORCH_MODEL
}

seq2seq_test_config = {
    'USER_IDS': TEST_IDS,
    'DATAFRAME_PATH': DATAFRAME_PATH,
    'CATEGORY': CATEGORY,
    'ANNOTATIONS': ANNOTATIONS,
    'PYTORCH_MODEL': PYTORCH_MODEL
}

In [8]:
# dataset
dataset = {}
dataset['train'] = load_data.VGGishEmbeddingDataset(seq2seq_train_config)
dataset['test'] = load_data.VGGishEmbeddingDataset(seq2seq_test_config)

In [9]:
# model
model = LSTMClassifier(input_dim = 128, hidden_dim = 16, output_dim = 2, num_layers = 1)
optimizer = optim.SGD(model.parameters(), lr = 0.5)
criterion = nn.CrossEntropyLoss()

In [10]:
# logging
print (f"{len(TRAIN_IDS)} training samples: {TRAIN_IDS[:5]}...")
print (f"{ len(TEST_IDS)} testing samples : {TEST_IDS[:5]}...")

64 training samples: [1927, 1878, 1882, 1933, 1898]...
16 testing samples : [1806, 1828, 1829, 1832, 1834]...


In [11]:
# training
for epoch in range(3):
    print ("Epoch : {}".format(epoch))
    train(
        dataset = dataset['train'],
        batch_size = 5,
        model = model,
        optimizer = optimizer,
        criterion = criterion
    )
    
    test(
        dataset = dataset['test'],
        model = model,
        criterion = criterion
    )

Epoch : 0
Batch : 1 / Loss : 0.6701251864433289
Batch : 2 / Loss : 0.49074020981788635
Batch : 3 / Loss : 0.49806687235832214
Batch : 4 / Loss : 0.44269075989723206
Batch : 5 / Loss : 0.5323425531387329
Batch : 6 / Loss : 0.4936315715312958
Batch : 7 / Loss : 0.5447154641151428
Batch : 8 / Loss : 0.4220910966396332
Batch : 9 / Loss : 0.3825794756412506
Batch : 10 / Loss : 0.31893038749694824
Batch : 11 / Loss : 0.46033066511154175
Batch : 12 / Loss : 0.3522374629974365
Batch : 13 / Loss : 0.49719366431236267
Test loss: 0.45826074481010437
Epoch : 1
Batch : 1 / Loss : 0.3080466091632843
Batch : 2 / Loss : 0.34028053283691406
Batch : 3 / Loss : 0.4222303330898285
Batch : 4 / Loss : 0.35527747869491577
Batch : 5 / Loss : 0.46489012241363525
Batch : 6 / Loss : 0.40947678685188293
Batch : 7 / Loss : 0.45276832580566406
Batch : 8 / Loss : 0.34383994340896606
Batch : 9 / Loss : 0.31809279322624207
Batch : 10 / Loss : 0.2623659372329712
Batch : 11 / Loss : 0.3975023627281189
Batch : 12 / Loss 

In [12]:
# save
timestr = time.strftime("%Y%m%d-%H%M%S")
torch.save(model, f"../seq2seq-{timestr}.pt")

In [15]:
model = torch.load("../seq2seq-20210107-203845.pt")

In [16]:
test_ids = [idx for idx in ANNOTATIONS.keys() if idx not in TRAIN_IDS]
test_ids = test_ids[:30]

# eval
predictions = np.array([])
labels = np.array([])

eval_config = {
    'USER_IDS': [],
    'DATAFRAME_PATH': DATAFRAME_PATH,
    'CATEGORY': CATEGORY,
    'ANNOTATIONS': ANNOTATIONS,
    'PYTORCH_MODEL': PYTORCH_MODEL
}

for use_i in test_ids:
    eval_config['USER_IDS'] = [use_i]
    feature, label = load_data.VGGishEmbeddingDataset(eval_config)[0]
    shape = feature.shape
    ypred = model(feature.view(shape[0], 1, shape[1])).squeeze(dim=0)
    m = nn.Softmax(dim=1)
    prediction = (m(ypred)[:, 1] > THRESHOLD).long()
    
    predictions = np.concatenate([predictions, prediction.numpy()])
    labels = np.concatenate([labels, label.numpy()])

print (classification_report(y_true=labels, y_pred=predictions))

              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95      2756
         1.0       0.72      0.71      0.71       483

    accuracy                           0.92      3239
   macro avg       0.83      0.83      0.83      3239
weighted avg       0.91      0.92      0.91      3239



In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true=labels, y_pred=predictions)